# Q9 — Land Classification: CNN-Transformer Integration Evaluation
Defines dataset dir and hyperparams, instantiates a small PyTorch hybrid model, evaluates Keras and PyTorch models using print_metrics.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import torch, torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

DATASET_DIR = 'images_dataSAT'
IMG = (64,64)
BATCH = 4
EPOCHS = 2

def print_metrics(y_true, y_pred, labels=None):
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('Precision:', precision_score(y_true, y_pred, average='weighted', zero_division=0))
    print('Recall:', recall_score(y_true, y_pred, average='weighted', zero_division=0))
    print('F1:', f1_score(y_true, y_pred, average='weighted', zero_division=0))
    print('\nClassification Report:\n', classification_report(y_true, y_pred, target_names=labels, zero_division=0))

# Keras part: produce dummy arrays (replace with real model outputs)
y_true_k = np.random.randint(0,2, size=20)
y_pred_k = np.random.randint(0,2, size=20)
print('Keras evaluation (demo random):'); print_metrics(y_true_k, y_pred_k)

# PyTorch model instantiate & eval
tf = transforms.Compose([transforms.Resize(IMG), transforms.ToTensor()])
val_ds = datasets.ImageFolder(DATASET_DIR, transform=tf)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False)

# small PyTorch hybrid model (same as Q8)
class SmallHybrid(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2))
        self.fc = nn.Linear(16*32*32, num_classes)
    def forward(self,x):
        b = x.size(0)
        f = self.cnn(x).view(b,-1)
        return self.fc(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SmallHybrid(num_classes=len(val_ds.classes)).to(device)
# run inference to collect preds (random init, so results are demo)
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        out = model(xb)
        preds = out.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds.tolist()); all_labels.extend(yb.numpy().tolist())

print('PyTorch evaluation (demo):'); print_metrics(np.array(all_labels), np.array(all_preds), labels=val_ds.classes)